#### If running in Google Colab, use the below route

+  the below path should be changed to the exact path that you save this script


In [0]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/PLearning/Precision Learning-PSU Only/precisionLearning/code/audio_features')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data_base_dir = '/content/drive/My Drive/PLearning/Precision Learning-PSU Only/precisionLearning/code/audio_features'


In [0]:
!pip install soundfile
!pip install pydub
!pip install SpeechRecognition

In [0]:
import soundfile as sf
from os import path
from pydub import AudioSegment
from pydub.silence import split_on_silence
import math
import re
import glob
import pandas as pd
import librosa
import numpy as np
import speech_recognition as sr
import string

In [0]:
#Driver code
#Load .wav files
import os 
import glob
audiodir=os.path.join(data_base_dir,'Extracted_audios')

print(audiodir)
audioFiles=[]
for file in os.listdir(audiodir):
    # print(file)
    if file.endswith('.wav'):
        afile=os.path.join(audiodir,file)
        audioFiles.append(afile)
print(len(audioFiles))
audioFiles[:3]

/content/drive/My Drive/Extracted_audios
184


['/content/drive/My Drive/Extracted_audios/7.NS.A.1b _ 7.NS.A.1c - Addition and Subtraction of Rational Numbers (Part 1 of 2)-LXZqE_soFzY.wav',
 '/content/drive/My Drive/Extracted_audios/CCSS 7.NS.1.d-k2rrJDcOGsw.wav',
 '/content/drive/My Drive/Extracted_audios/[7.EE.4a-1.0] Equations in Form px + q = r - Common Core Standard-i1quLORCZ2g.wav']

In [0]:
def pitch_tuning(audioFiles):
    output = []
    for i in range(len(audioFiles)): 
        temp = []
        y,sr = librosa.load(audioFiles[i])
        pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
        # Select out pitches with high energy
        pitches = pitches[magnitudes > np.median(magnitudes)]
        pitch = librosa.pitch_tuning(pitches)
        name = audioFiles[i][len(audioFiles[i])-15:-4]
        temp.append(name)
        temp.append(pitch)
        output.append(temp)
    df = pd.DataFrame(data=output, columns=['Video ids', 'pitch_tuning'])
    from pathlib import Path
    data_dir=Path(os.path.join(data_base_dir,'new_output'))
    data_dir.mkdir(exist_ok=True)
    df.to_csv('{}/pitch_tuning.csv'.format(data_dir), index=False)

In [0]:
# Find high pitches in the audio
pitch_tuning(audioFiles)

In [0]:
def noise(audioFiles,data_base_dir):
    output = []
    # audioFiles = glob.glob("*.wav")
    # output = []
    for i in range(len(audioFiles)):
        count = 0
        temp =[]
        y,sr = librosa.load(audioFiles[i])
        flatness = librosa.feature.spectral_flatness(y=y)
        # Select out noise with high spectral flatness
        noise =np.median(flatness)
        count  = flatness[flatness>=1.0]
        noisecount = len(count)/np.size(flatness)
        name = audioFiles[i][len(audioFiles[i])-15:-4]
        temp.append(name)
        temp.append(noise)
        temp.append(noisecount)
        output.append(np.transpose(temp))
    df = pd.DataFrame(data=output, columns=['Video ids', 'median flatness', 'count of 1s/total frames'])
    #Outputs the noisy values in the audio
    from pathlib import Path
    data_dir=Path(os.path.join(data_base_dir,'new_output'))
    data_dir.mkdir(exist_ok=True)
    df.to_csv('{}/noise.csv'.format(data_dir), index=False)

In [0]:
#Find noise in the audio
noise(audioFiles,data_base_dir)

In [0]:
def speed(audioFiles,data_base_dir):
    output= []
    for i in range(len(audioFiles)):  
        audio = AudioSegment.from_wav(audioFiles[i])
        # Split audio by each minute
        duration = len(audio)
        number_of_segments = duration/(60*1000)
        k = math.floor(number_of_segments)
        # k is the number of segments in the audio
        if k == 0:
            k=1
        print(audioFiles[i])
        t1 = 1
        s = 0
        t2=60 *1000
        temp = []
        for j in range(1,k+1):
            t2 = j * 60 * 1000
            newaudio = audio[t1:t2]
            # Path to save the split audio files
            filename = str(audioFiles[i][:-4])+'_'+str(j)+'.wav'
            newaudio.export(filename, format="wav")
            #Speech recognition using Google Speech API
            r = sr.Recognizer()
            with sr.AudioFile(filename) as source:
                readaudio = r.record(source)
                try:
                    test = r.recognize_google(readaudio)
                    res = sum([i.strip(string.punctuation).isalpha() for i in test.split()])
                except sr.UnknownValueError:
                    #Incase of an error an arbitrary value is assigned
                    print("Google Speech Recognition could not understand audio")
                    res = 50
            s =s + res
            t1 = t1+ (60 *1000)
        # Same method for the remainder seconds
        if (t2<duration):
            k = k+1
            newaudio = audio[t2+1:duration]
            filename = str(audioFiles[i][:-4])+'_'+str(j)+'.wav'
            newaudio.export(filename, format="wav")
            r = sr.Recognizer()
            with sr.AudioFile(filename) as source:
                readaudio = r.record(source)
                try:
                    test = r.recognize_google(readaudio)
                    res = sum([i.strip(string.punctuation).isalpha() for i in test.split()])
                except sr.UnknownValueError:
                    print("Google Speech Recognition could not understand audio")
                    res = 50 
            s =s + res
        #Name denotes the video ids in the audio file name
        name = audioFiles[i][len(audioFiles[i])-15:-4]
        temp.append(name)
        wpm = s/k
        temp.append(wpm)
        output.append(temp)
    output = np.transpose(output)
    output = np.transpose(output)
    df = pd.DataFrame(data = output, columns=['Video ids', 'wpm'] )
    # Outputs the average words per minute in the audio
    from pathlib import Path
    data_dir=Path(os.path.join(data_base_dir,'new_output'))
    data_dir.mkdir(exist_ok=True)
    df.to_csv('{}/wpm.csv'.format(data_dir), index=False)

In [0]:
#Audio files will be split and saved to the same folder as the original files. Make a copy of the original files to rerun this code in
import os

os.chdir( '/content/drive/My Drive')
speed(audioFiles,data_base_dir)

/content/drive/My Drive/Extracted_audios/7.NS.A.1b _ 7.NS.A.1c - Addition and Subtraction of Rational Numbers (Part 1 of 2)-LXZqE_soFzY.wav
/content/drive/My Drive/Extracted_audios/CCSS 7.NS.1.d-k2rrJDcOGsw.wav
